In [1]:
from pianoq.misc.mplc_lab_imports import *  

dir_path = r'G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror'
if not os.path.exists(dir_path):
    os.mkdir(dir_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Camera things

In [44]:
cam = PCOCamera()
cam.set_exposure_time(0.5)

# ml = MPLCAligner(use_cam=False)
# ml.update('1to5w4f', '5to11w8', pi_steps_x=[0], pi_steps_plane=1)

In [50]:
cam.set_exposure_time(2)

In [47]:
roi=(750, 350, 1200, 850)
live_cam(cam, roi=roi)

In [48]:
mplc.load_masks(masks)
roi=(750, 350, 1200, 850)
cam.show_image(roi=roi)

(<Figure size 640x480 with 2 Axes>, <Axes: >)

In [60]:
roi=(700, 300, 1250, 900)
name = f'tilts_{tilts}.fits'
cam.save_image(path=rf'{dir_path}\{name}', roi=roi)
cam.show_image(roi=roi)

(<Figure size 640x480 with 2 Axes>, <Axes: >)

In [ ]:
timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
path = rf'{dir_path}\{timestamp}_{name}.cam'
cam.save_image(path)

In [61]:
cam.close()

# MPLC 

In [3]:
modes_to_keep = np.array([3, 8, 13, 18, 23, 28, 33, 38, 43, 48])

tilts = None 
mplc = MPLCDevice(linear_tilts=True, plane_10_tilts=tilts)
mplc.restore_location()
print('Got MPLC')
# masks_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_09_10_14_00_34_old.masks"
masks_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_09_25_13_19_15_extra_1cm.masks"
# masks_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_17_first_diffuser\2024_09_17_12_19_58_diffuser_d1d.masks"
# masks_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_09_29_14_27_10_U1U.masks"  # with dead zone 
# masks_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_09_30_08_56_10_U1U.masks"  # with dead zone 
# masks_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_10_fixing_phases_different_Us\U1U_2024_09_10_14_00_34.masks"  # old masks  

msks = MPLCMasks()
msks.loadfrom(masks_path)
masks = msks.real_masks
masks = remove_input_modes(masks, modes_to_keep=modes_to_keep)
# phases = np.random.uniform(0, 2*np.pi, size=50)
# masks = add_phase_input_spots(masks, phases)
mplc.load_masks(masks)
# mplc.load_masks(masks, linear_tilts=True, plane_10_tilts=None)
mplc.restore_location()

Got MPLC


# hardware 

In [4]:
backlash = 0.0
wait_after_move = 0.3
mxs, mys = get_signal_motors(backlash=backlash, wait_after_move=wait_after_move)
mxi, myi = get_idler_motors(backlash=backlash, wait_after_move=wait_after_move)
tt = get_timetagger(integration_time=1)

Got Zaber motors
Got Thorlabs motors


In [37]:
mxi.move_absolute(9.405)
myi.move_absolute(4.795)
print(mxi.get_position())
print(myi.get_position())

9.405
4.795


In [25]:
mxs.move_absolute(12.91)
mys.move_absolute(5.72)
print(mxs.get_position())
print(mys.get_position())

12.909994500000002
5.720000625


In [147]:
myi.move_relative(0.01)

In [4]:
print(mxi.get_position()) # 9.2701
print(myi.get_position()) # 3.3100

9.3
4.05


BS
- best idler: ~9K
- best signal: ~8.3K 

Mirror:
- best idler: 
- best signal: 


# New WFM 

## with defocus distance and dead zone 

In [6]:
all_Us_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_10_fixing_phases_different_Us\for_ronen_Haar_800.mat"
all_Us = loadmat(all_Us_path)['U1_all']
U_no = 1 
dir_path = r'G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror'
timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
masks_path =    fr'{dir_path}\{timestamp}_U1U.masks'
mplc_res_path = fr'{dir_path}\{timestamp}_U1U.mplcsim'
mplc_sim = create_WFM_unitary_masks(all_Us[:, :, U_no], N_iterations=15, out_path=masks_path, dead_middle_zone=450e-6, last_plane_extra_dist=8.4e-3)
mplc_sim.res.saveto(mplc_res_path, smaller=True)

{'wavelength': 8.1e-07, 'dist_after_plane': array([0.087 , 0.087 , 0.087 , 0.087 , 0.138 , 0.087 , 0.087 , 0.087 ,
       0.087 , 0.0954]), 'N_iterations': 15, 'Nx': 140, 'Ny': 360, 'dx': 1.25e-05, 'dy': 1.25e-05, 'max_k_constraint': 0.15, 'min_log_level': 2, 'size_factor': 3, 'use_mask_offset': True, 'symmetric_masks': False, 'active_planes': array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True]), 'N_modes': 10}
1


100%|██████████| 15/15 [02:45<00:00, 11.01s/it]


In [7]:
mplc_sim.res.show_overlap()

In [8]:
mplc_sim.res._calc_fidelity()
mplc_sim.res.fidelity

0.9896838134848069

In [9]:
mplc_sim.res.show_all()

# mark spot locations 

In [11]:
from pianoq_results.scan_result import ScanResult
timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
locs_signal_path = fr'{dir_path}\{timestamp}_s2_sig_mirror_3.5.locs'
locs_idler_path = fr'{dir_path}\{timestamp}_s1_idl_mirror_3.5.locs'

res_idl = ScanResult(fr'{dir_path}\2024_10_06_12_06_17_idler_3_5_spots.scan')
res_sig = ScanResult(fr'{dir_path}\2024_10_06_12_07_31_signal_3_5_spots.scan')
# res_idl.show_singles()
# res_sig.show_singles()
res_idl.get_xys(1, saveto_path=locs_idler_path)
res_sig.get_xys(2, saveto_path=locs_signal_path)

Saved to G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_10_06_12_25_13_s1_idl_mirror_3.5.locs
Saved to G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_10_06_12_25_13_s2_sig_mirror_3.5.locs


[(12.984470529470531, 7.019139194139194),
 (12.948240093240095, 6.8358558108558105),
 (12.937584082584083, 6.462895437895438),
 (12.897091242091243, 6.077147852147852),
 (12.869385614385616, 5.697793872793873)]

# Phases

In [38]:
phases_path = fr'{dir_path}\{tnow()}_fine_new_mirror.phases'
integration_time = 30

phases_result = PhaseFinderResult(fr"{dir_path}\2024_10_07_11_42_08_fine_new_mirror.phases")

pf = PhaseFinder(mplc=mplc, integration_time=integration_time, remote_tagger=True, saveto_path=phases_path,
                 modes_to_keep=modes_to_keep, intial_phases=phases_result.phases, coin_window=tt.coin_window, no_hw_mode=True)

pf.time_tagger = tt
pf.time_tagger.set_integration_time(integration_time)

# locs 
locs_sig = np.load(fr'{dir_path}\2024_10_06_12_25_13_s2_sig_mirror_3.5.locs')['locs']
locs_idl = np.load(fr'{dir_path}\2024_10_06_12_25_13_s1_idl_mirror_3.5.locs')['locs']

# i,j of corr matrix that is supposed to be strong 
i = 2
j = 2
mxi.move_absolute(locs_idl[i, 0])
myi.move_absolute(locs_idl[i, 1])
mxs.move_absolute(locs_sig[j, 0])
mys.move_absolute(locs_sig[j, 1])        
mplc.restore_location()
time.sleep(1)
pf.find_phases()

0,0: 19402.9, 14293.9, 4.2
0,1: 19277.6, 14190.3, 3.9
0,2: 19367.2, 14345.3, 4.6
0,3: 19487.0, 14342.3, 4.5
0,4: 19750.9, 14301.5, 5.2
0,5: 19728.0, 14416.7, 5.5
0,6: 19821.5, 14479.6, 5.6
0,7: 19808.7, 14558.3, 5.8
0,8: 19838.2, 14519.5, 5.1
0,9: 19773.6, 13719.9, 5.3
1,0: 19764.6, 14496.2, 6.1
1,1: 19815.0, 14602.3, 5.7
1,2: 19819.9, 14636.6, 4.3
1,3: 19775.5, 14572.5, 3.1
1,4: 19765.3, 13739.8, 2.4
1,5: 19752.4, 13694.7, 1.3
1,6: 19867.5, 14601.9, 1.5
1,7: 19808.7, 14544.1, 2.5
1,8: 19854.9, 14449.0, 4.1
1,9: 19804.2, 14528.3, 5.9
2,0: 19846.2, 14564.2, 6.5
2,1: 19826.0, 14501.4, 5.8
2,2: 19833.2, 14233.5, 6.1
2,3: 19832.8, 14372.3, 4.7
2,4: 19869.6, 14364.3, 4.3
2,5: 19853.7, 14458.9, 4.7
2,6: 19863.9, 14562.0, 5.2
2,7: 19880.7, 14736.5, 4.8
2,8: 19892.2, 14194.9, 5.4
2,9: 19894.3, 14286.4, 5.8
3,0: 19938.1, 14578.8, 7.1
3,1: 19930.4, 14663.3, 5.3
3,2: 19897.7, 14704.3, 4.3
3,3: 19909.8, 14481.3, 3.2
3,4: 19906.2, 14093.1, 1.0
3,5: 19940.8, 13199.6, 1.4
3,6: 19941.0, 12682.3, 2.0
3

# Correlations

In [56]:
backlash = 0.0
wait_after_move = 0.3
integration_time = 5
coin_window = 1.2e-9

masks_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_09_25_13_19_15_extra_1cm.masks"
phases_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_10_07_12_01_25_fine_new_mirror.phases"
masks = get_good_masks(masks_path=masks_path, modes_to_keep=modes_to_keep, phases_path=phases_path,)
# mplc.load_masks(masks)

scan_path = fr'{dir_path}\{tnow()}.dscan'

locs_idl = np.load(fr'{dir_path}\2024_10_07_10_45_00_fine_s1_idl_mirror_3_5_spots.locs')['locs']
locs_sig = np.load(fr'{dir_path}\2024_10_07_10_45_00_fine_s2_sig_mirror_3_5_spots.locs')['locs']

dps = DiscretePhotonScanner(locs_sig, locs_idl, integration_time=integration_time, remote_tagger=True, saveto_path=scan_path,
                            backlash=backlash, wait_after_move=wait_after_move, coin_window=coin_window, no_hw_mode=True)

dps.m_sig_x, dps.m_sig_y, dps.m_idl_x, dps.m_idl_y, dps.time_tagger = mxs, mys, mxi, myi, tt
dps.time_tagger.set_integration_time(integration_time)
time.sleep(1)
dps.scan()

beginning scan
0, 0: 16057.00, 11075.60, 4.00
0, 1: 16093.60, 13543.80, 1.40
0, 2: 16053.60, 13871.20, 1.40
0, 3: 15932.40, 8827.80, 0.80
0, 4: 15945.60, 1878.20, 0.00
1, 0: 18640.40, 11120.80, 1.60
1, 1: 18569.40, 13690.20, 7.40
1, 2: 18589.00, 13867.00, 1.00
1, 3: 18607.40, 8739.80, 1.20
1, 4: 18564.80, 1879.80, 0.20
2, 0: 17984.80, 11068.20, 2.60
2, 1: 17946.60, 13597.40, 1.60
2, 2: 17987.20, 13914.80, 7.00
2, 3: 17925.20, 8792.80, 0.20
2, 4: 17860.00, 1866.40, 0.00
3, 0: 8786.00, 11135.20, 0.20
3, 1: 8781.00, 13730.20, 0.20
3, 2: 8856.20, 13996.40, 0.60
3, 3: 8758.40, 8884.60, 1.00
3, 4: 8731.60, 1904.80, 0.00
4, 0: 3195.20, 11123.40, 0.20
4, 1: 3202.00, 13732.00, 0.00
4, 2: 3158.80, 13918.20, 0.20
4, 3: 3191.80, 8828.20, 0.00
4, 4: 3249.40, 1878.40, 0.00


In [57]:
dps.res.show_singles()
dps.res.show()
dps.res.real_coins.sum()

25.156527033088

In [52]:
mplc.close()